# import

In [288]:
import pandas as pd

# 보통 현재 작업 폴더(CWD)가 src 이므로 부모(프로젝트 루트)를 sys.path에 추가
import sys, os
from pathlib import Path

ROOT = Path.cwd().parent
if (ROOT / "utils").exists():
    sys.path.insert(0, str(ROOT))
else:
    # 혹시 구조가 더 깊을 때 대비
    ROOT2 = ROOT.parent
    if (ROOT2 / "utils").exists():
        sys.path.insert(0, str(ROOT2))

from utils.angular_util import compute_joint_angles, compute_connected_unit_vectors, flatten_vectors

## 수집한 데이터 합치기

In [289]:
HAND_COUNT = 21 * 3
POSE_COUNT = 11 * 3

## 헤더 추가
🚨 주의!!!! 이미 헤더가 있는 경우 또 추가될 수 있음!

In [290]:
# for i in range(7, 8):
#     data = pd.read_csv(f"../../data/sign_data/sign_data_{i}.csv")
    # data.insert(0, 'label', [i] * len(data))
    # columns = [i for i in range(len(data.columns) - 1)]
    # columns.insert(0, 'label')
    # data.columns = columns
    # data.to_csv(f"../../data/sign_data/sign_data_{i}.csv", index=False)

In [291]:
# i = 5
# data = pd.read_csv(f"../../data/sign_data/sign_data_{i}.csv", index_col=False)
# data.loc[:, 'label'] = i
# data.to_csv(f"../../data/sign_data/sign_data_{i}.csv", index=False)

## 병합

In [292]:
merged_data = pd.DataFrame()

for i, label in enumerate([0, 1, 2, 3, 4, 5, 6, 14]):
    print(label)
    data = pd.read_csv(f"../../data/sign_data/sign_data_{label}.csv", index_col=False)
    data = data.drop(['label'], axis=1)
    data.loc[:, 'label'] = i
    data = data.sample(n = 500, replace=False)
    merged_data = pd.concat([merged_data, data], ignore_index=True)

df = merged_data

0
1
2
3
4
5
6
14


In [293]:
# merged_data.to_csv("../../data/merge_num_7_data.csv", index=False)

## 데이터 불러오기

In [294]:
# import pandas as pd
# df = pd.read_csv("../../data/merge_num_7_data.csv")

## 클래스 확인

In [295]:
df['label'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int64)

## x, y 데이터

In [296]:
y = df['label']
x = df.drop(['label'], axis=1)

print(x.shape, y.shape)

(4000, 159) (4000,)


# 데이터 전처리

In [297]:
right_points_x = df.iloc[:, 64:64+63]
face_points_x = df.iloc[:, 64+63:]
nose_point_x = face_points_x.iloc[:, 0]
print(right_points_x.shape, face_points_x.shape, nose_point_x.shape)

(4000, 63) (4000, 33) (4000,)


In [298]:
angles_list = []
vector_list = []
for i in range(len(right_points_x)):
    row = right_points_x.iloc[i].tolist()
    angles, _, _ = compute_joint_angles(row)
    vector, _ = compute_connected_unit_vectors(row)
    vector = flatten_vectors(vector)
    vector_list.append(vector)
    angles_list.append(angles)

print(len(angles_list[0]), len(vector_list[0]))

angular_x = pd.DataFrame(angles_list, columns=None)
vector_x = pd.DataFrame(vector_list, columns=None)

print(angular_x.shape, vector_x.shape)

15 60
(4000, 15) (4000, 60)


In [299]:
result_x = pd.concat([angular_x, vector_x], axis=1)
result_x.columns = range(len(result_x.columns))

print(result_x.shape)

(4000, 75)


In [300]:
# result_x.to_csv("../../data/angle_vector_num_7_data.csv", index=False)

## 데이터 분할하기

In [301]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(result_x, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

## 데이터 스케일링

In [302]:
# from sklearn.preprocessing import MinMaxScaler
# mms = MinMaxScaler()
# x_train = mms.fit_transform(x_train)
# x_test = mms.transform(x_test)

## 모델 학습

In [303]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np

model = XGBClassifier(n_estimators=200, max_depth=3, learning_rate=0.1)
#model = RandomForestClassifier(n_estimators=300, max_depth=5, random_state=42)
#model = LogisticRegression(solver='liblinear', max_iter=100)
model.fit(x_train, y_train)

score = model.score(x_test, y_test)

print(f"Accuracy: {np.round(score*100, 2)}%")

Accuracy: 99.0%


In [304]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.99      0.98      0.98       100
           1       0.99      0.99      0.99       100
           2       0.98      0.97      0.97       100
           3       0.96      0.99      0.98       100
           4       1.00      0.99      0.99       100
           5       1.00      1.00      1.00       100
           6       1.00      1.00      1.00       100
           7       1.00      1.00      1.00       100

    accuracy                           0.99       800
   macro avg       0.99      0.99      0.99       800
weighted avg       0.99      0.99      0.99       800



## 모델 저장하기

In [305]:
import joblib

joblib.dump(model, "../../models/xgb_sample_angle_vector_model.pkl")

['../../models/xgb_sample_angle_vector_model.pkl']